## Drivers of PE-Score (Figure 5-7)

In [1]:
dir_path = '/Users/jeremyzyang/Library/CloudStorage/GoogleDrive-jeremy.z.yang@gmail.com/My Drive/tiktok_revision'
setwd(dir_path)

In [2]:
load('engagement_that_sells.RData')
options(warn = -1)

In [3]:
suppressWarnings(suppressMessages(library(data.table)))
suppressWarnings(suppressMessages(library(tidyverse)))
suppressWarnings(suppressMessages(library(lfe)))
suppressWarnings(suppressMessages(library(patchwork)))
suppressWarnings(suppressMessages(library(zoo)))
suppressWarnings(suppressMessages(library(doParallel)))
suppressWarnings(suppressMessages(library(staggered)))
suppressWarnings(suppressMessages(library(fixest)))
suppressWarnings(suppressMessages(library(faux)))
suppressWarnings(suppressMessages(library(stargazer)))
suppressWarnings(suppressMessages(library(xgboost)))
suppressWarnings(suppressMessages(library(caret)))

In [4]:
sessionInfo()

R version 4.2.1 (2022-06-23)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.2/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.2/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] caret_6.0-93      lattice_0.20-45   xgboost_1.6.0.1   stargazer_5.2.3  
 [5] faux_1.1.0        fixest_0.10.4     staggered_1.1     doParallel_1.0.17
 [9] iterators_1.0.14  foreach_1.5.2     zoo_1.8-11        patchwork_1.1.2  
[13] lfe_2.8-8         Matrix_1.5-1      forcats_0.5.2     stringr_1.4.1    
[17] dplyr_1.0.10      purrr_0.3.4       readr_2.1.3       tidyr_1.2.1      
[21] tibble_3.1.8      ggplot2_3.3.6     tidyverse_1.3.2   data.table_1.14.2

loaded via a names

In [5]:
ls()

[1] "dir_path"           "model_construction" "model_evaluation"  
[4] "search"

In [6]:
training = model_construction
df = model_evaluation

#### Figure 5 (to-do)

#### Figure 6 (to-do)

#### Figure 7 (to-do)